In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!unzip "/content/drive/My Drive/img_align_celeba.zip"
!cat '/content/drive/My Drive/utils.py'
!cat '/content/drive/My Drive/ops.py'
!cat '/content/drive/My Drive/models_64x64.py'
import sys
sys.path.append('/content/drive/My Drive')

In [ ]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import glob
import utils
import traceback
import numpy as np
import tensorflow as tf
import models_64x64 as models


""" param """
epoch = 50
batch_size = 64
lr = 0.0002
z_dim = 100
gpu_id = 3

''' data '''
# you should prepare your own data in ./data/img_align_celeba
# celeba original size is [218, 178, 3]


def preprocess_fn(img):
    crop_size = 108
    re_size = 64
    img = tf.image.crop_to_bounding_box(img, (218 - crop_size) // 2, (178 - crop_size) // 2, crop_size, crop_size)
    img = tf.to_float(tf.image.resize_images(img, [re_size, re_size], method=tf.image.ResizeMethod.BICUBIC)) / 127.5 - 1
    return img

img_paths = glob.glob('/content/img_align_celeba/')
data_pool = utils.DiskImageData('/content/img_align_celeba/', batch_size, shape=[218, 178, 3], preprocess_fn=preprocess_fn)

tf.reset_default_graph()

""" graphs """
with tf.device('/gpu:%d' % gpu_id):
    ''' models '''
    generator = models.generator
    discriminator = models.discriminator

    ''' graph '''
    # inputs
    real = tf.placeholder(tf.float32, shape=[None, 64, 64, 3])
    z = tf.placeholder(tf.float32, shape=[None, z_dim])

    # generate
    fake = generator(z, reuse=False)

    # dicriminate
    r_logit = discriminator(real, reuse=False)
    f_logit = discriminator(fake)

    # losses
    d_r_loss = tf.losses.mean_squared_error(tf.ones_like(r_logit), r_logit)
    d_f_loss = tf.losses.mean_squared_error(tf.zeros_like(f_logit), f_logit)
    d_loss = (d_r_loss + d_f_loss) / 2.0
    g_loss = tf.losses.mean_squared_error(tf.ones_like(f_logit), f_logit)

    # otpims
    d_var = utils.trainable_variables('discriminator')
    g_var = utils.trainable_variables('generator')
    d_step = tf.train.AdamOptimizer(learning_rate=lr, beta1=0.5).minimize(d_loss, var_list=d_var)
    g_step = tf.train.AdamOptimizer(learning_rate=lr, beta1=0.5).minimize(g_loss, var_list=g_var)

    # summaries
    d_summary = utils.summary({d_loss: 'd_loss'})
    g_summary = utils.summary({g_loss: 'g_loss'})

    # sample
    f_sample = generator(z, training=False)


""" train """
''' init '''
# session
sess = utils.session()
# iteration counter
it_cnt, update_cnt = utils.counter()
# saver
saver = tf.train.Saver(max_to_keep=5)
# summary writer
summary_writer = tf.summary.FileWriter('/content/drive/My Drive/GP/summaries/celeba_lsgan', sess.graph)

''' initialization '''
ckpt_dir = '/content/drive/My Drive/GP/checkpoints/celeba_lsgan'
utils.mkdir(ckpt_dir + '/')
if not utils.load_checkpoint(ckpt_dir, sess):
    sess.run(tf.global_variables_initializer())

''' train '''
try:
    z_ipt_sample = np.random.normal(size=[100, z_dim])

    batch_epoch = len(data_pool) // (batch_size)
    max_it = epoch * batch_epoch
    for it in range(sess.run(it_cnt), max_it):
        sess.run(update_cnt)

        # which epoch
        epoch = it // batch_epoch
        it_epoch = it % batch_epoch + 1

        # batch data
        real_ipt = data_pool.batch()
        z_ipt = np.random.normal(size=[batch_size, z_dim])

        # train D
        d_summary_opt, _ = sess.run([d_summary, d_step], feed_dict={real: real_ipt, z: z_ipt})
        summary_writer.add_summary(d_summary_opt, it)

        # train G
        sess.run([g_step], feed_dict={z: z_ipt})
        g_summary_opt, _ = sess.run([g_summary, g_step], feed_dict={z: z_ipt})
        summary_writer.add_summary(g_summary_opt, it)

        # display
        if it % 1 == 0:
            print("Epoch: (%3d) (%5d/%5d)" % (epoch, it_epoch, batch_epoch))

        # save
        if (it + 1) % 1000 == 0:
            save_path = saver.save(sess, '%s/Epoch_(%d)_(%dof%d).ckpt' % (ckpt_dir, epoch, it_epoch, batch_epoch))
            print('Model saved in file: % s' % save_path)

        # sample
        if (it + 1) % 100 == 0:
            f_sample_opt = sess.run(f_sample, feed_dict={z: z_ipt_sample})

            save_dir = '/content/drive/My Drive/GP/sample_images_while_training/celeba_lsgan'
            utils.mkdir(save_dir + '/')
            utils.imwrite(utils.immerge(f_sample_opt, 10, 10), '%s/Epoch_(%d)_(%dof%d).jpg' % (save_dir, epoch, it_epoch, batch_epoch))

except Exception as e:
    traceback.print_exc()
finally:
    print(" [*] Close main session!")
    sess.close()


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

#Data is  202599
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(tuple(tensor_list)).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.range(limit).shuffle(limit).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuf

Epoch: (  0) (  101/ 3165)
Epoch: (  0) (  102/ 3165)
Epoch: (  0) (  103/ 3165)
Epoch: (  0) (  104/ 3165)
Epoch: (  0) (  105/ 3165)
Epoch: (  0) (  106/ 3165)
Epoch: (  0) (  107/ 3165)
Epoch: (  0) (  108/ 3165)
Epoch: (  0) (  109/ 3165)
Epoch: (  0) (  110/ 3165)
Epoch: (  0) (  111/ 3165)
Epoch: (  0) (  112/ 3165)
Epoch: (  0) (  113/ 3165)
Epoch: (  0) (  114/ 3165)
Epoch: (  0) (  115/ 3165)
Epoch: (  0) (  116/ 3165)
Epoch: (  0) (  117/ 3165)
Epoch: (  0) (  118/ 3165)
Epoch: (  0) (  119/ 3165)
Epoch: (  0) (  120/ 3165)
Epoch: (  0) (  121/ 3165)
Epoch: (  0) (  122/ 3165)
Epoch: (  0) (  123/ 3165)
Epoch: (  0) (  124/ 3165)
Epoch: (  0) (  125/ 3165)
Epoch: (  0) (  126/ 3165)
Epoch: (  0) (  127/ 3165)
Epoch: (  0) (  128/ 3165)
Epoch: (  0) (  129/ 3165)
Epoch: (  0) (  130/ 3165)
Epoch: (  0) (  131/ 3165)
Epoch: (  0) (  132/ 3165)
Epoch: (  0) (  133/ 3165)
Epoch: (  0) (  134/ 3165)
Epoch: (  0) (  135/ 3165)
Epoch: (  0) (  136/ 3165)
Epoch: (  0) (  137/ 3165)
E

Epoch: (  0) (  200/ 3165)
Epoch: (  0) (  201/ 3165)
Epoch: (  0) (  202/ 3165)
Epoch: (  0) (  203/ 3165)
Epoch: (  0) (  204/ 3165)
Epoch: (  0) (  205/ 3165)
Epoch: (  0) (  206/ 3165)
Epoch: (  0) (  207/ 3165)
Epoch: (  0) (  208/ 3165)
Epoch: (  0) (  209/ 3165)
Epoch: (  0) (  210/ 3165)
Epoch: (  0) (  211/ 3165)
Epoch: (  0) (  212/ 3165)
Epoch: (  0) (  213/ 3165)
Epoch: (  0) (  214/ 3165)
Epoch: (  0) (  215/ 3165)
Epoch: (  0) (  216/ 3165)
Epoch: (  0) (  217/ 3165)
Epoch: (  0) (  218/ 3165)
Epoch: (  0) (  219/ 3165)
Epoch: (  0) (  220/ 3165)
Epoch: (  0) (  221/ 3165)
Epoch: (  0) (  222/ 3165)
Epoch: (  0) (  223/ 3165)
Epoch: (  0) (  224/ 3165)
Epoch: (  0) (  225/ 3165)
Epoch: (  0) (  226/ 3165)
Epoch: (  0) (  227/ 3165)
Epoch: (  0) (  228/ 3165)
Epoch: (  0) (  229/ 3165)
Epoch: (  0) (  230/ 3165)
Epoch: (  0) (  231/ 3165)
Epoch: (  0) (  232/ 3165)
Epoch: (  0) (  233/ 3165)
Epoch: (  0) (  234/ 3165)
Epoch: (  0) (  235/ 3165)
Epoch: (  0) (  236/ 3165)
E

Epoch: (  0) (  300/ 3165)
Epoch: (  0) (  301/ 3165)
Epoch: (  0) (  302/ 3165)
Epoch: (  0) (  303/ 3165)
Epoch: (  0) (  304/ 3165)
Epoch: (  0) (  305/ 3165)
Epoch: (  0) (  306/ 3165)
Epoch: (  0) (  307/ 3165)
Epoch: (  0) (  308/ 3165)
Epoch: (  0) (  309/ 3165)
Epoch: (  0) (  310/ 3165)
Epoch: (  0) (  311/ 3165)
Epoch: (  0) (  312/ 3165)
Epoch: (  0) (  313/ 3165)
Epoch: (  0) (  314/ 3165)
Epoch: (  0) (  315/ 3165)
Epoch: (  0) (  316/ 3165)
Epoch: (  0) (  317/ 3165)
Epoch: (  0) (  318/ 3165)
Epoch: (  0) (  319/ 3165)
Epoch: (  0) (  320/ 3165)
Epoch: (  0) (  321/ 3165)
Epoch: (  0) (  322/ 3165)
Epoch: (  0) (  323/ 3165)
Epoch: (  0) (  324/ 3165)
Epoch: (  0) (  325/ 3165)
Epoch: (  0) (  326/ 3165)
Epoch: (  0) (  327/ 3165)
Epoch: (  0) (  328/ 3165)
Epoch: (  0) (  329/ 3165)
Epoch: (  0) (  330/ 3165)
Epoch: (  0) (  331/ 3165)
Epoch: (  0) (  332/ 3165)
Epoch: (  0) (  333/ 3165)
Epoch: (  0) (  334/ 3165)
Epoch: (  0) (  335/ 3165)
Epoch: (  0) (  336/ 3165)
E

Epoch: (  0) (  400/ 3165)
Epoch: (  0) (  401/ 3165)
Epoch: (  0) (  402/ 3165)
Epoch: (  0) (  403/ 3165)
Epoch: (  0) (  404/ 3165)
Epoch: (  0) (  405/ 3165)
Epoch: (  0) (  406/ 3165)
Epoch: (  0) (  407/ 3165)
Epoch: (  0) (  408/ 3165)
Epoch: (  0) (  409/ 3165)
Epoch: (  0) (  410/ 3165)
Epoch: (  0) (  411/ 3165)
Epoch: (  0) (  412/ 3165)
Epoch: (  0) (  413/ 3165)
Epoch: (  0) (  414/ 3165)
Epoch: (  0) (  415/ 3165)
Epoch: (  0) (  416/ 3165)
Epoch: (  0) (  417/ 3165)
Epoch: (  0) (  418/ 3165)
Epoch: (  0) (  419/ 3165)
Epoch: (  0) (  420/ 3165)
Epoch: (  0) (  421/ 3165)
Epoch: (  0) (  422/ 3165)
Epoch: (  0) (  423/ 3165)
Epoch: (  0) (  424/ 3165)
Epoch: (  0) (  425/ 3165)
Epoch: (  0) (  426/ 3165)
Epoch: (  0) (  427/ 3165)
Epoch: (  0) (  428/ 3165)
Epoch: (  0) (  429/ 3165)
Epoch: (  0) (  430/ 3165)
Epoch: (  0) (  431/ 3165)
Epoch: (  0) (  432/ 3165)
Epoch: (  0) (  433/ 3165)
Epoch: (  0) (  434/ 3165)
Epoch: (  0) (  435/ 3165)
Epoch: (  0) (  436/ 3165)
E

...

Epoch: ( 18) ( 1530/ 3165)
Epoch: ( 18) ( 1531/ 3165)
Epoch: ( 18) ( 1532/ 3165)
Epoch: ( 18) ( 1533/ 3165)
Epoch: ( 18) ( 1534/ 3165)
Epoch: ( 18) ( 1535/ 3165)
Epoch: ( 18) ( 1536/ 3165)
Epoch: ( 18) ( 1537/ 3165)
Epoch: ( 18) ( 1538/ 3165)
Epoch: ( 18) ( 1539/ 3165)
Epoch: ( 18) ( 1540/ 3165)
Epoch: ( 18) ( 1541/ 3165)
Epoch: ( 18) ( 1542/ 3165)
Epoch: ( 18) ( 1543/ 3165)
Epoch: ( 18) ( 1544/ 3165)
Epoch: ( 18) ( 1545/ 3165)
Epoch: ( 18) ( 1546/ 3165)
Epoch: ( 18) ( 1547/ 3165)
Epoch: ( 18) ( 1548/ 3165)
Epoch: ( 18) ( 1549/ 3165)
Epoch: ( 18) ( 1550/ 3165)
Epoch: ( 18) ( 1551/ 3165)
Epoch: ( 18) ( 1552/ 3165)
Epoch: ( 18) ( 1553/ 3165)
Epoch: ( 18) ( 1554/ 3165)
Epoch: ( 18) ( 1555/ 3165)
Epoch: ( 18) ( 1556/ 3165)
Epoch: ( 18) ( 1557/ 3165)
Epoch: ( 18) ( 1558/ 3165)
Epoch: ( 18) ( 1559/ 3165)
Epoch: ( 18) ( 1560/ 3165)
Epoch: ( 18) ( 1561/ 3165)
Epoch: ( 18) ( 1562/ 3165)
Epoch: ( 18) ( 1563/ 3165)
Epoch: ( 18) ( 1564/ 3165)
Epoch: ( 18) ( 1565/ 3165)
Epoch: ( 18) ( 1566/ 3165)
E

Epoch: ( 18) ( 1630/ 3165)
Epoch: ( 18) ( 1631/ 3165)
Epoch: ( 18) ( 1632/ 3165)
Epoch: ( 18) ( 1633/ 3165)
Epoch: ( 18) ( 1634/ 3165)
Epoch: ( 18) ( 1635/ 3165)
Epoch: ( 18) ( 1636/ 3165)
Epoch: ( 18) ( 1637/ 3165)
Epoch: ( 18) ( 1638/ 3165)
Epoch: ( 18) ( 1639/ 3165)
Epoch: ( 18) ( 1640/ 3165)
Epoch: ( 18) ( 1641/ 3165)
Epoch: ( 18) ( 1642/ 3165)
Epoch: ( 18) ( 1643/ 3165)
Epoch: ( 18) ( 1644/ 3165)
Epoch: ( 18) ( 1645/ 3165)
Epoch: ( 18) ( 1646/ 3165)
Epoch: ( 18) ( 1647/ 3165)
Epoch: ( 18) ( 1648/ 3165)
Epoch: ( 18) ( 1649/ 3165)
Epoch: ( 18) ( 1650/ 3165)
Epoch: ( 18) ( 1651/ 3165)
Epoch: ( 18) ( 1652/ 3165)
Epoch: ( 18) ( 1653/ 3165)
Epoch: ( 18) ( 1654/ 3165)
Epoch: ( 18) ( 1655/ 3165)
Epoch: ( 18) ( 1656/ 3165)
Epoch: ( 18) ( 1657/ 3165)
Epoch: ( 18) ( 1658/ 3165)
Epoch: ( 18) ( 1659/ 3165)
Epoch: ( 18) ( 1660/ 3165)
Epoch: ( 18) ( 1661/ 3165)
Epoch: ( 18) ( 1662/ 3165)
Epoch: ( 18) ( 1663/ 3165)
Epoch: ( 18) ( 1664/ 3165)
Epoch: ( 18) ( 1665/ 3165)
Epoch: ( 18) ( 1666/ 3165)
E

Epoch: ( 18) ( 1730/ 3165)
Epoch: ( 18) ( 1731/ 3165)
Epoch: ( 18) ( 1732/ 3165)
Epoch: ( 18) ( 1733/ 3165)
Epoch: ( 18) ( 1734/ 3165)
Epoch: ( 18) ( 1735/ 3165)
Epoch: ( 18) ( 1736/ 3165)
Epoch: ( 18) ( 1737/ 3165)
Epoch: ( 18) ( 1738/ 3165)
Epoch: ( 18) ( 1739/ 3165)
Epoch: ( 18) ( 1740/ 3165)
Epoch: ( 18) ( 1741/ 3165)
Epoch: ( 18) ( 1742/ 3165)
Epoch: ( 18) ( 1743/ 3165)
Epoch: ( 18) ( 1744/ 3165)
Epoch: ( 18) ( 1745/ 3165)
Epoch: ( 18) ( 1746/ 3165)
Epoch: ( 18) ( 1747/ 3165)
Epoch: ( 18) ( 1748/ 3165)
Epoch: ( 18) ( 1749/ 3165)
Epoch: ( 18) ( 1750/ 3165)
Epoch: ( 18) ( 1751/ 3165)
Epoch: ( 18) ( 1752/ 3165)
Epoch: ( 18) ( 1753/ 3165)
Epoch: ( 18) ( 1754/ 3165)
Epoch: ( 18) ( 1755/ 3165)
Epoch: ( 18) ( 1756/ 3165)
Epoch: ( 18) ( 1757/ 3165)
Epoch: ( 18) ( 1758/ 3165)
Epoch: ( 18) ( 1759/ 3165)
Epoch: ( 18) ( 1760/ 3165)
Epoch: ( 18) ( 1761/ 3165)
Epoch: ( 18) ( 1762/ 3165)
Epoch: ( 18) ( 1763/ 3165)
Epoch: ( 18) ( 1764/ 3165)
Epoch: ( 18) ( 1765/ 3165)
Epoch: ( 18) ( 1766/ 3165)
E

Epoch: ( 18) ( 1830/ 3165)
Epoch: ( 18) ( 1831/ 3165)
Epoch: ( 18) ( 1832/ 3165)
Epoch: ( 18) ( 1833/ 3165)
Epoch: ( 18) ( 1834/ 3165)
Epoch: ( 18) ( 1835/ 3165)
Epoch: ( 18) ( 1836/ 3165)
Epoch: ( 18) ( 1837/ 3165)
Epoch: ( 18) ( 1838/ 3165)
Epoch: ( 18) ( 1839/ 3165)
Epoch: ( 18) ( 1840/ 3165)
Epoch: ( 18) ( 1841/ 3165)
Epoch: ( 18) ( 1842/ 3165)
Epoch: ( 18) ( 1843/ 3165)
Epoch: ( 18) ( 1844/ 3165)
Epoch: ( 18) ( 1845/ 3165)
Epoch: ( 18) ( 1846/ 3165)
Epoch: ( 18) ( 1847/ 3165)
Epoch: ( 18) ( 1848/ 3165)
Epoch: ( 18) ( 1849/ 3165)
Epoch: ( 18) ( 1850/ 3165)
Epoch: ( 18) ( 1851/ 3165)
Epoch: ( 18) ( 1852/ 3165)
Epoch: ( 18) ( 1853/ 3165)
Epoch: ( 18) ( 1854/ 3165)
Epoch: ( 18) ( 1855/ 3165)
Epoch: ( 18) ( 1856/ 3165)
Epoch: ( 18) ( 1857/ 3165)
Epoch: ( 18) ( 1858/ 3165)
Epoch: ( 18) ( 1859/ 3165)
Epoch: ( 18) ( 1860/ 3165)
Epoch: ( 18) ( 1861/ 3165)
Epoch: ( 18) ( 1862/ 3165)
Epoch: ( 18) ( 1863/ 3165)
Epoch: ( 18) ( 1864/ 3165)
Epoch: ( 18) ( 1865/ 3165)
Epoch: ( 18) ( 1866/ 3165)
E

Epoch: ( 18) ( 1930/ 3165)
Epoch: ( 18) ( 1931/ 3165)
Epoch: ( 18) ( 1932/ 3165)
Epoch: ( 18) ( 1933/ 3165)
Epoch: ( 18) ( 1934/ 3165)
Epoch: ( 18) ( 1935/ 3165)
Epoch: ( 18) ( 1936/ 3165)
Epoch: ( 18) ( 1937/ 3165)
Epoch: ( 18) ( 1938/ 3165)
Epoch: ( 18) ( 1939/ 3165)
Epoch: ( 18) ( 1940/ 3165)
Epoch: ( 18) ( 1941/ 3165)
Epoch: ( 18) ( 1942/ 3165)
Epoch: ( 18) ( 1943/ 3165)
Epoch: ( 18) ( 1944/ 3165)
Epoch: ( 18) ( 1945/ 3165)
Epoch: ( 18) ( 1946/ 3165)
Epoch: ( 18) ( 1947/ 3165)
Epoch: ( 18) ( 1948/ 3165)
Epoch: ( 18) ( 1949/ 3165)
Epoch: ( 18) ( 1950/ 3165)
Epoch: ( 18) ( 1951/ 3165)
Epoch: ( 18) ( 1952/ 3165)
Epoch: ( 18) ( 1953/ 3165)
Epoch: ( 18) ( 1954/ 3165)
Epoch: ( 18) ( 1955/ 3165)
Epoch: ( 18) ( 1956/ 3165)
Epoch: ( 18) ( 1957/ 3165)
Epoch: ( 18) ( 1958/ 3165)
Epoch: ( 18) ( 1959/ 3165)
Epoch: ( 18) ( 1960/ 3165)
Epoch: ( 18) ( 1961/ 3165)
Epoch: ( 18) ( 1962/ 3165)
Epoch: ( 18) ( 1963/ 3165)
Epoch: ( 18) ( 1964/ 3165)
Epoch: ( 18) ( 1965/ 3165)
Epoch: ( 18) ( 1966/ 3165)
E